### Mathematical Model for Multi-Commodity Flow Problem

In [3]:
# Solve a multi-commodity flow problem.  Two products ('Pencils' and 'Pens') are produced in 2 cities 
# ('Detroit' and 'Denver') and must be sent to warehouses in 3 cities ('Boston', 'New York', and 'Seattle') 
# to satisfy supply/demand ('inflow[h,i]').

# Flows on the transportation network must respect arc capacity constraints ('capacity[i,j]'). 

# The objective is to minimize the sum of the arc transportation costs ('cost[i,j]').

In [4]:
 # First, import packages
import pandas as pd
from IPython.display import Image 
import gurobipy as gp
from gurobipy import GRB

# Define a gurobipy model for the decision problem
m = gp.Model('netflow')

### Data

In [5]:
# Base data
commodities = ["Pencils", "Pens"]
nodes = ["Detroit", "Denver", "Boston", "New York", "Seattle"]

arcs, capacity = gp.multidict(
    {
        ("Detroit", "Boston"): 100,
        ("Detroit", "New York"): 80,
        ("Detroit", "Seattle"): 120,
        ("Denver", "Boston"): 120,
        ("Denver", "New York"): 120,
        ("Denver", "Seattle"): 120,
    }
)

In [6]:
# Cost for triplets commodity-source-destination
cost = {
    ("Pencils", "Detroit", "Boston"): 10,
    ("Pencils", "Detroit", "New York"): 20,
    ("Pencils", "Detroit", "Seattle"): 60,
    ("Pencils", "Denver", "Boston"): 40,
    ("Pencils", "Denver", "New York"): 40,
    ("Pencils", "Denver", "Seattle"): 30,
    ("Pens", "Detroit", "Boston"): 20,
    ("Pens", "Detroit", "New York"): 20,
    ("Pens", "Detroit", "Seattle"): 80,
    ("Pens", "Denver", "Boston"): 60,
    ("Pens", "Denver", "New York"): 70,
    ("Pens", "Denver", "Seattle"): 30,
}

In [7]:
# Supply (> 0) and demand (< 0) for pairs of commodity-city
inflow = {
    ("Pencils", "Detroit"): 50,
    ("Pencils", "Denver"): 60,
    ("Pencils", "Boston"): -50,
    ("Pencils", "New York"): -50,
    ("Pencils", "Seattle"): -10,
    ("Pens", "Detroit"): 60,
    ("Pens", "Denver"): 40,
    ("Pens", "Boston"): -40,
    ("Pens", "New York"): -30,
    ("Pens", "Seattle"): -30,
}

# Total supply for pencils: Detroit(50) + Denver(60) = 110
# Total supply for pens:    Detroit(60) + Denver(40) = 100

# Total demand for pencils: Bostoon(50) + New York(50) + Seattle(10) = 110
# Total demand for pens:    Bostoon(40) + New York(30) + Seattle(30) = 100

### Variables

In [8]:
# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

# Alternate version 01:
#flow = m.addVars(commodities, arcs, obj=cost, name="flow")

### Constraints

In [9]:
# Arc capacity constraint
c1 = m.addConstrs((gp.quicksum(flow[h, i, j] for h in commodities) <= capacity[i, j] for i,j in arcs))

c1
# Alternate version 01:
#m.addConstrs((flow.sum("*", i, j) <= capacity[i, j] for i, j in arcs), "cap")

# Alternate version 02:
# Equivalent version using Python looping
# for i, j in arcs:
#   m.addConstr(sum(flow[h, i, j] for h in commodities) <= capacity[i, j],
#               "cap[%s, %s]" % (i, j))

{('Detroit', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [10]:
# # Flow-conservation constraints
c2 = m.addConstrs(
   (gp.quicksum(flow[h, i, j] for i, j in arcs) + inflow[h, j] ==
     (gp.quicksum(flow[h, j, k] for j, k in arcs)) for h in commodities for j in nodes), "node")

m.update()
c2

# Alternate version 01:
#m.addConstrs(
#    (
#        flow.sum(h, "*", j) + inflow[h, j] == flow.sum(h, j, "*")
#        for h in commodities
#        for j in nodes
#    ),
#    "node",
#)

# Alternate version 02:
# m.addConstrs(
#   (gp.quicksum(flow[h, i, j] for i, j in arcs.select('*', j)) + inflow[h, j] ==
#     gp.quicksum(flow[h, j, k] for j, k in arcs.select(j, '*'))
#     for h in commodities for j in nodes), "node")

{('Pencils', 'Detroit'): <gurobi.Constr node[Pencils,Detroit]>,
 ('Pencils', 'Denver'): <gurobi.Constr node[Pencils,Denver]>,
 ('Pencils', 'Boston'): <gurobi.Constr node[Pencils,Boston]>,
 ('Pencils', 'New York'): <gurobi.Constr node[Pencils,New York]>,
 ('Pencils', 'Seattle'): <gurobi.Constr node[Pencils,Seattle]>,
 ('Pens', 'Detroit'): <gurobi.Constr node[Pens,Detroit]>,
 ('Pens', 'Denver'): <gurobi.Constr node[Pens,Denver]>,
 ('Pens', 'Boston'): <gurobi.Constr node[Pens,Boston]>,
 ('Pens', 'New York'): <gurobi.Constr node[Pens,New York]>,
 ('Pens', 'Seattle'): <gurobi.Constr node[Pens,Seattle]>}

### Objective Function 

In [12]:
m.setObjective(gp.quicksum(cost[h, i, j]*flow[h, i, j] for h in commodities for i, j in arcs), GRB.MINIMIZE)

### Optimal Solution

In [13]:
# Compute optimal solution
m.update()
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 19.6.0 19H2026)

CPU model: Intel(R) Core(TM) i7-3520M CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 16 rows, 12 columns and 12 nonzeros
Model fingerprint: 0x4d877562
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve time: 0.04s

Solved in 0 iterations and 0.04 seconds (0.00 work units)
Infeasible model


### Print Solution

In [ ]:

# Print solution
if m.Status == GRB.OPTIMAL:
    solution = m.getAttr("X", flow)
    for h in commodities:
        print(f"\nOptimal flows for {h}:")
        for i, j in arcs:
            if solution[h, i, j] > 0:
                print(f"{i} -> {j}: {solution[h, i, j]:g}")